Blending IPython's widgets and mpld3's plugins
==============================================

This notebook performs a function quite similar to the 'sliderPlugin' example.
Browser side visualisation is actionable and triggers recalculations in the ipython backend.
While the sliderPlugin connects to the kernel, we use IPython's facilities : interact does the lifting for us.

Because you need an IPython instance running, you cannot use it directly on nbviewer for example. _You have to download this notebook and run it in IPython yourself._ 

I used IPython 3.0.0-dev as of 2014/11/03. The widget interface does not seems so stable for now so you may have to tinker to get this working. If you experience problems I think that the examples we built on would be good material to get the whole thing working again.

Objective
---------

We want to fit a curve in a cloud of points.
The points are drag/drop-able by the user of the notebook and upon dropping the point, the fit is recalculated.

The model can be pretty much any $R \to R$ function, with any number of parameters.

In what follows you will see it :
- (partially) applied to an "first order exponential response to an Heavyside function" (for lack of better wording on my side);
- applied to an arc-tangente.


Architecture
------------

Here is how things are organized :
0. code copyied from the ClickInfo/DragPoints examples on the mpld3 side will generate updates when the user drag and drop the circles;
1. these updates are the new coordinates of a given point of the cloud;
2. the update trigger the 'change' event on a text widget from IPython (code taken from the custom widget example);
3. IPython cogs and wheels transmit the update back to the IPython server side;
4. where we recalculate parameters, and redraw everything.

In [13]:
# imports widget side
# see https://github.com/ipython/ipython/blob/2.x/examples/Interactive%20Widgets/Custom%20Widgets.ipynb
# and https://github.com/ipython/ipython/blob/master/examples/Interactive%20Widgets/Custom%20Widget%20-%20Hello%20World.ipynb

from __future__ import print_function # For py 2.7 compat

from IPython.html import widgets # Widget definitions
from IPython.display import display # Used to display widgets in the notebook
from IPython.utils.traitlets import Unicode # Used to declare attributes of our widget
from IPython.html.widgets import interact, interactive, fixed

In [14]:
# imports render side
# see http://mpld3.github.io/examples/drag_points.html

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import mpld3
from mpld3 import plugins, utils

In [15]:
# imports solve side
# see http://stackoverflow.com/questions/8739227/how-to-solve-a-pair-of-nonlinear-equations-using-python

from scipy.optimize import fsolve

#def expchelon(a, b, x):
#    return a * (1 - np.exp(-b * x))

#def fun(p1, p2):
#    x1, y1 = p1
#    x2, y2 = p2
#    def equations(p):
#        a, b = p
#        return (y1 - expchelon(a, b, x1), y2 - expchelon(a, b, x2))
#    return equations

#equations = fun((1,1), (2,4))
#a, b =  fsolve(equations, (1, 1))

#print((a, b), expchelon(a, b, 1), expchelon(a, b, 2))

In [16]:
# widget sync'd python side
class GraphWidget(widgets.DOMWidget):
    _view_name = Unicode('GraphView', sync=True)
    description = 'coord'    
    value = Unicode(sync=True)

In [17]:
%%javascript
//widget javascript side
require(["widgets/js/widget", "widgets/js/manager"], function(widget, manager){
    // is based on the DatePickerView
    var GraphView = widget.DOMWidgetView.extend({
        render: function() {
            //@ attr id : this is the id we reach to in the dragended function in the DragPlugin
            this.$text = $('<input />')
                .attr('type', 'text')
                .attr('id', 'feedback_widget')                
                .appendTo(this.$el);
        },
        
        update: function() {
            this.$text.val(this.model.get('value'));
            return GraphView.__super__.update.apply(this);
        },
        
        events: {"change": "handle_change"},
        
        handle_change: function(event) {
            this.model.set('value', this.$text.val());
            this.touch();
        },
    });
    
    manager.WidgetManager.register_widget_view('GraphView', GraphView);
});

<IPython.core.display.Javascript object>

In [18]:
# visu plugin
# based on DragPlugin
class DragPlugin(plugins.PluginBase):
    JAVASCRIPT = r"""
$('#feedback_widget').hide();
mpld3.register_plugin("drag", DragPlugin);
DragPlugin.prototype = Object.create(mpld3.Plugin.prototype);
DragPlugin.prototype.constructor = DragPlugin;
DragPlugin.prototype.requiredProps = ["id"];
DragPlugin.prototype.defaultProps = {}
function DragPlugin(fig, props){
    mpld3.Plugin.call(this, fig, props);
    mpld3.insert_css("#" + fig.figid + " path.dragging",
                     {"fill-opacity": "1.0 !important",
                      "stroke-opacity": "1.0 !important"});
};$

DragPlugin.prototype.draw = function(){
    var obj = mpld3.get_element(this.props.id);

    var drag = d3.behavior.drag()
        .origin(function(d) { return {x:obj.ax.x(d[0]),
                                      y:obj.ax.y(d[1])}; })
        .on("dragstart", dragstarted)
        .on("drag", dragged)
        .on("dragend", dragended);

    obj.elements()
       .data(obj.offsets)
       .style("cursor", "default")
       .call(drag);

    function dragstarted(d) {
      d3.event.sourceEvent.stopPropagation();
      d3.select(this).classed("dragging", true);
    }

    function dragged(d, i) {
      d[0] = obj.ax.x.invert(d3.event.x);
      d[1] = obj.ax.y.invert(d3.event.y);
      d3.select(this)
        .attr("transform", "translate(" + [d3.event.x,d3.event.y] + ")");
    }

    function dragended(d,i) {
      d3.select(this).classed("dragging", false);
      // feed back the new position to python, triggering 'change' on the widget
      $('#feedback_widget').val("" + i + "," + d[0] + "," + d[1]).trigger("change");
    }
}"""

    def __init__(self, points):
        if isinstance(points, mpl.lines.Line2D):
            suffix = "pts"
        else:
            suffix = None

        self.dict_ = {"type": "drag",
                      "id": utils.get_id(points, suffix)}

In [19]:
# fit and draw
class Fit(object):
    def __init__(self, simulate, double_seeding=False):
        self.simulate = simulate
         
        # i will draw initial points at random
        # the number of points will increase until we match arity with the function to be fit(ted?)
        pseudo_fit = []
        while len(pseudo_fit) < 100:
            # just in case, I want to avoid inifite loops...
            try:
                simulate(0, pseudo_fit)
                print("we have %d parameters"%len(pseudo_fit))
                break
            except IndexError:
                pseudo_fit.append(1)
                
        # we generate a random cloud 
        # the dots are distributed in (>0, >0) quadrant    
        self.p = np.random.standard_exponential((len(pseudo_fit), 2))
        
        # first guess ! all ones.
        self.fit = np.array(pseudo_fit)
                
    def make_equations(self):
        def equations(params):
            return self.p[:,1] - self.simulate(self.p[:,0], params)
        self.equations = equations
    
    def recalc_param(self):
        self.make_equations()
        self.fit = fsolve(self.equations, np.ones(self.fit.shape), xtol=0.01)
        
    def redraw(self, coord):
        # we have an update !
        
        # record the new position for given point 
        if coord != "":
            i, x, y = coord.split(",")
            i = int(i)
            self.p[i][0] = float(x)
            self.p[i][1] = float(y)
            
        # recalculate best fit
        self.recalc_param()
        
        # draw things
        x = np.linspace(0, 10, 50) # 50 x points from 0 to 10
        y = self.simulate(x, self.fit)
    
        fig, ax = plt.subplots()

        points = ax.plot(self.p[:,0], self.p[:,1],'or', alpha=0.5, markersize=10, markeredgewidth=1)
        
        ax.plot(x,y,'r-')
        ax.set_title("Click and Drag\n, we match on : %s"%np.array_str(self.fit, precision=2), fontsize=12)

        plugins.connect(fig, DragPlugin(points[0]))

        fig_h = mpld3.display()
        display(fig_h)

In [20]:
# click and drag not active here, we just show how we fit

def exp_ech(x, params):
    return params[0] * (1 - np.exp(-params[1] * x))

# we ensure we will fit nicely by setting p[0] at [0,0]
# in effect adding one degree of liberty
Fit(exp_ech).redraw("0,0,0")

we have 2 parameters


In [21]:
def arctan(x, params):
    return params[0] * np.arctan(params[1] * x + params[2])

my_fit = Fit(arctan)

# not sure why, but you can't do
# interact(my_fit.redraw, coord=GraphWidget())
# so we need :
def f(coord):
    return my_fit.redraw(coord)
    
interact(f, coord=GraphWidget())

we have 3 parameters


ValueError: GraphWidget() cannot be transformed to a widget

In [26]:
from bokeh.sampledata.autompg import autompg as df
import bokeh 



# input options
hist = Histogram(df['mpg'], title="df['mpg']")
hist2 = Histogram(df, 'displ', title="df, 'displ'")
hist3 = Histogram(df, values='hp', title="df, values='hp'")

hist4 = Histogram(df, values='hp', color='cyl',
                  title="df, values='hp', color='cyl'", legend='top_right')

hist5 = Histogram(df, values='mpg', bins=50,
                  title="df, values='mpg', bins=50")

output_file("histograms.html")

show(
    vplot(
        hplot(hist, hist2, hist3),
        hplot(hist4, hist5)
    )
)

NameError: name 'Histogram' is not defined

In [29]:

import numpy as np
import scipy.special

from bokeh.plotting import figure, show, output_file

p1 = figure(title="Normal Distribution (μ=0, σ=0.5)",tools="save",
            background_fill_color="#E8DDCB")

mu, sigma = 0, 0.5

measured = np.random.normal(mu, sigma, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(-2, 2, 1000)
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
p1.line(x, pdf, line_color="#D95B43", line_width=8, alpha=0.7, legend="PDF")
p1.line(x, cdf, line_color="white", line_width=2, alpha=0.7, legend="CDF")

p1.legend.location = "top_left"
p1.xaxis.axis_label = 'x'
p1.yaxis.axis_label = 'Pr(x)'



p2 = figure(title="Log Normal Distribution (μ=0, σ=0.5)", tools="save",
            background_fill_color="#E8DDCB")

mu, sigma = 0, 0.5

measured = np.random.lognormal(mu, sigma, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(0, 8.0, 1000)
pdf = 1/(x* sigma * np.sqrt(2*np.pi)) * np.exp(-(np.log(x)-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((np.log(x)-mu)/(np.sqrt(2)*sigma)))/2

p2.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
p2.line(x, pdf, line_color="#D95B43", line_width=8, alpha=0.7, legend="PDF")
p2.line(x, cdf, line_color="white", line_width=2, alpha=0.7, legend="CDF")

p2.legend.location = "bottom_right"
p2.xaxis.axis_label = 'x'
p2.yaxis.axis_label = 'Pr(x)'



p3 = figure(title="Gamma Distribution (k=1, θ=2)", tools="save",
            background_fill_color="#E8DDCB")

k, theta = 1.0, 2.0

measured = np.random.gamma(k, theta, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(0, 20.0, 1000)
pdf = x**(k-1) * np.exp(-x/theta) / (theta**k * scipy.special.gamma(k))
cdf = scipy.special.gammainc(k, x/theta) / scipy.special.gamma(k)

p3.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
p3.line(x, pdf, line_color="#D95B43", line_width=8, alpha=0.7, legend="PDF")
p3.line(x, cdf, line_color="white", line_width=2, alpha=0.7, legend="CDF")

p3.legend.location = "top_left"
p3.xaxis.axis_label = 'x'
p3.yaxis.axis_label = 'Pr(x)'



p4 = figure(title="Beta Distribution (α=2, β=2)", tools="save",
            background_fill_color="#E8DDCB")

alpha, beta = 2.0, 2.0

measured = np.random.beta(alpha, beta, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(0, 1, 1000)
pdf = x**(alpha-1) * (1-x)**(beta-1) / scipy.special.beta(alpha, beta)
cdf = scipy.special.btdtr(alpha, beta, x)

p4.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
p4.line(x, pdf, line_color="#D95B43", line_width=8, alpha=0.7, legend="PDF")
p4.line(x, cdf, line_color="white", line_width=2, alpha=0.7, legend="CDF")

p4.xaxis.axis_label = 'x'
p4.yaxis.axis_label = 'Pr(x)'



p5 = figure(title="Weibull Distribution (λ=1, k=1.25)", tools="save",
            background_fill_color="#E8DDCB")

lam, k = 1, 1.25

measured = lam*(-np.log(np.random.uniform(0, 1, 1000)))**(1/k)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(0, 8, 1000)
pdf = (k/lam)*(x/lam)**(k-1) * np.exp(-(x/lam)**k)
cdf = 1 - np.exp(-(x/lam)**k)

p5.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
       fill_color="#036564", line_color="#033649")
p5.line(x, pdf, line_color="#D95B43", line_width=8, alpha=0.7, legend="PDF")
p5.line(x, cdf, line_color="white", line_width=2, alpha=0.7, legend="CDF")

p5.legend.location = "top_left"
p5.xaxis.axis_label = 'x'
p5.yaxis.axis_label = 'Pr(x)'



output_file('histogram.html', title="histogram.py example")

show(vplot(p1,p2,p3,p4,p5))

/Users/deepak/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:39: RuntimeWarning: divide by zero encountered in true_divide
/Users/deepak/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:39: RuntimeWarning: divide by zero encountered in log
/Users/deepak/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:39: RuntimeWarning: invalid value encountered in multiply
/Users/deepak/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:40: RuntimeWarning: divide by zero encountered in log


NameError: name 'vplot' is not defined

In [31]:
import bokeh

In [33]:
from Bokeh.ipynb import Area, show, output_file

# create some example data
data = dict(
    python=[2, 3, 7, 5, 26, 221, 44, 233, 254, 265, 266, 267, 120, 111],
    pypy=[12, 33, 47, 15, 126, 121, 144, 233, 254, 225, 226, 267, 110, 130],
    jython=[22, 43, 10, 25, 26, 101, 114, 203, 194, 215, 201, 227, 139, 160],
)

area = Area(data, title="Area Chart", legend="top_left",
            xlabel='time', ylabel='memory')

output_file('area.html')
show(area)

ImportError: cannot import name 'Area'

In [37]:
from bokeh.charts import Histogram, show, output_notebook

p = Histogram(df, values='score', color = 'month',
      title="Histograms for two different months",
      legend='top_right', bins=10, palette=['blue', 'orange'])
show(p)

ModuleNotFoundError: No module named 'bokeh.charts'

In [36]:

''' Present an interactive function explorer with slider widgets.
Scrub the sliders to change the properties of the ``sin`` curve, or
type into the title text box to update the title of the plot.
Use the ``bokeh serve`` command to run the example by executing:
    bokeh serve sliders.py
at your command prompt. Then navigate to the URL
    http://localhost:5006/sliders
in your browser.
'''
import numpy as np

from bokeh.io import curdoc
from bokeh.layouts import row, widgetbox
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, TextInput
from bokeh.plotting import figure

# Set up data
N = 200
x = np.linspace(0, 4*np.pi, N)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y))


# Set up plot
plot = figure(plot_height=400, plot_width=400, title="my sine wave",
              tools="crosshair,pan,reset,save,wheel_zoom",
              x_range=[0, 4*np.pi], y_range=[-2.5, 2.5])

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)


# Set up widgets
text = TextInput(title="title", value='my sine wave')
offset = Slider(title="offset", value=0.0, start=-5.0, end=5.0, step=0.1)
amplitude = Slider(title="amplitude", value=1.0, start=-5.0, end=5.0, step=0.1)
phase = Slider(title="phase", value=0.0, start=0.0, end=2*np.pi)
freq = Slider(title="frequency", value=1.0, start=0.1, end=5.1, step=0.1)


# Set up callbacks
def update_title(attrname, old, new):
    plot.title.text = text.value

text.on_change('value', update_title)

def update_data(attrname, old, new):

    # Get the current slider values
    a = amplitude.value
    b = offset.value
    w = phase.value
    k = freq.value

    # Generate the new curve
    x = np.linspace(0, 4*np.pi, N)
    y = a*np.sin(k*x + w) + b

    source.data = dict(x=x, y=y)

for w in [offset, amplitude, phase, freq]:
    w.on_change('value', update_data)


# Set up layouts and add to document
inputs = widgetbox(text, offset, amplitude, phase, freq)

curdoc().add_root(row(inputs, plot, width=800))
curdoc().title = "Sliders"

In [41]:
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure
from bokeh.transform import factor_cmap

output_file("bar_nested_colormapped.html")

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

palette = ["#c9d9d3", "#718dbf", "#e84d60"]

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (fruit, year) for fruit in fruits for year in years ]
counts = sum(zip(data['2015'], data['2016'], data['2017']), ()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), plot_height=350, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",
       fill_color=factor_cmap('x', palette=palette, factors=years, start=1, end=2))

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

In [40]:

from math import pi

import pandas as pd

from bokeh.plotting import figure, show, output_file
from bokeh.sampledata.stocks import MSFT

df = pd.DataFrame(MSFT)[:50]
df["date"] = pd.to_datetime(df["date"])

mids = (df.open + df.close)/2
spans = abs(df.close-df.open)

inc = df.close > df.open
dec = df.open > df.close
w = 12*60*60*1000 # half day in ms

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000, toolbar_location="left")

p.title = "MSFT Candlestick"
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3

p.segment(df.date, df.high, df.date, df.low, color="black")
p.rect(df.date[inc], mids[inc], w, spans[inc], fill_color="#D5E1DD", line_color="black")
p.rect(df.date[dec], mids[dec], w, spans[dec], fill_color="#F2583E", line_color="black")

output_file("candlestick.html", title="candlestick.py example")

show(p)  # open a browser

ValueError: expected an instance of type Title, got MSFT Candlestick of type str

In [2]:
from bokeh.charts import Bar, output_file, show
from bokeh.layouts import row

# best support is with data in a format that is table-like
data = {
    'sample': ['1st', '2nd', '1st', '2nd', '1st', '2nd'],
    'interpreter': ['python', 'python', 'pypy', 'pypy', 'jython', 'jython'],
    'timing': [-2, 5, 12, 40, 22, 30]
}

# x-axis labels pulled from the interpreter column, stacking labels from sample column
bar = Bar(data, values='timing', label='interpreter', stack='sample', agg='mean',
          title="Python Interpreter Sampling", legend='top_right', plot_width=400)

# table-like data results in reconfiguration of the chart with no data manipulation
bar2 = Bar(data, values='timing', label=['interpreter', 'sample'],
           agg='mean', title="Python Interpreters", plot_width=400)

output_file("stacked_bar.html")
show(row(bar, bar2))

ModuleNotFoundError: No module named 'bokeh.charts'

In [3]:
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure
from bokeh.transform import factor_cmap

output_file("bar_nested_colormapped.html")

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

palette = ["#c9d9d3", "#718dbf", "#e84d60"]

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (fruit, year) for fruit in fruits for year in years ]
counts = sum(zip(data['2015'], data['2016'], data['2017']), ()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), plot_height=350, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",
       fill_color=factor_cmap('x', palette=palette, factors=years, start=1, end=2))

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

In [5]:
from bokeh.plotting import figure, output_file, show

# prepare some data
x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

# create a new plot with a title and axis labels
p = figure(title="simple line example", x_axis_label='x', y_axis_label='y')

# add a line renderer with legend and line thickness
p.line(x, y, legend="Temp.", line_width=2)

# show the results
show(p)